In [2]:
'''
    purpose of this file is to preprocess the json file containing the recipes and
    to add the preprocessed files to the db
'''
import json

In [3]:
#opening the json file
with open('/home/shubham/JavaScript/side-project-shubham0831/recipes_with_nutritional_info.json') as json_file:
    data = json.load(json_file)

In [4]:
#random testing
print(len(data))
# test_data = data[1000]
# print(len(data))

# print('------')

# for key, value in test_data.items():
#     print(key)

51235


In [21]:
#recipe_list is a list containing dicts, each dict has a single recipe information
#each dict has a single recipe information, because mongoDB requires that, since each dict is a document
recipe_list = []

#extracting useful info from the data list
for dat in data:
    #following dict will contain the recipe info
    recipe_dict = dict()
    
    name = dat['title'] 
    ingredient_list = dat['ingredients']
    ingredient_unit = dat['unit']
    ingredient_quantity = dat['quantity']
    instruction_list = dat['instructions']
    nutrition_values = dat['nutr_values_per100g']
    instruction = ''

    ingredients = []
    ingredientQuantity = dict()
    # print(name)

    for i in range(len(ingredient_list)):
        item = ingredient_list[i]['text']
        quant = ingredient_quantity[i]['text']
        unit = ingredient_unit[i]['text']

        mainIngredient = item.split(',')[0]
        ingredients.append(mainIngredient)
        dose = quant + " " + unit
        ingredientQuantity[mainIngredient] = dose

    for inst in instruction_list:
        ins = inst['text']
        instruction += ins + "\n"

    recipe_dict['name'] = name
    recipe_dict['ingredients'] = ingredients
    recipe_dict['ingredient_quantity'] = ingredientQuantity
    recipe_dict['instructions'] = instruction
    recipe_dict['nutrition_values'] = nutrition_values
    
    recipe_list.append(recipe_dict)

In [22]:
#sample data
print(recipe_list[0])

{'name': 'Yogurt Parfaits', 'ingredients': ['yogurt', 'strawberries', 'cereals ready-to-eat'], 'ingredient_quantity': {'yogurt': '8 ounce', 'strawberries': '1 cup', 'cereals ready-to-eat': '1/4 cup'}, 'instructions': 'Layer all ingredients in a serving dish.\n', 'nutrition_values': {'energy': 81.12946131894766, 'fat': 2.140139263515891, 'protein': 6.914436593565536, 'salt': 0.05597816738985967, 'saturates': 0.36534716195613937, 'sugars': 5.08634103436144}}


In [23]:
#mongodb client for python
import pymongo

In [24]:
#reading configs
with open('config.json') as json_file:
    config = json.load(json_file)

username = config['username']
password = config['password']
database = config['database']

In [25]:
#connecting to db
uri = "mongodb+srv://" +username+":"+password+"@"+database+".tjeuf.mongodb.net/RecipeSearch?retryWrites=true&w=majority"
client = pymongo.MongoClient(uri)
db = client.Recipes
recipe_collection = db.Recipe

In [26]:
#inserting elements into db
recipe_collection.insert_many(recipe_list)

In [30]:
recipe_collection.update_many(
    { "ingredients": "spartan" },
    { "$set": { "ingredients.$": "chocolate chip" } }
)

In [37]:
recipe_collection.update_many(
    {"ingredient_quantity" : "$"},
    { "$rename": { 'spartan': 'chocolate chip'} } 
)


In [38]:
recipe_collection.update_many( { }, { "$rename": 
                                     { "ingredient_quantity.spartan": "ingredient_quantity.chocolate chip" } } )